# Topic Modeling

This is a fork of https://github.com/adashofdata/nlp-in-python-tutorial.  Rather than using the transcripts of stand-up comedians, it uses the text of New Testament books of the Bible.

- Created Nov 2022
- Revised Nov 2024

## Introduction

Another popular text analysis technique is called topic modeling. The ultimate goal of topic modeling is to find various topics that are present in your corpus. Each document in the corpus will be made up of at least one topic, if not multiple topics.

In this notebook, we will be covering the steps on how to do **Latent Dirichlet Allocation (LDA)**, which is one of many topic modeling techniques. It was specifically designed for text data.

To use a topic modeling technique, you need to provide (1) a document-term matrix and (2) the number of topics you would like the algorithm to pick up.

Once the topic modeling technique is applied, your job as a human is to interpret the results and see if the mix of words in each topic make sense. If they don't make sense, you can try changing up the number of topics, the terms in the document-term matrix, model parameters, or even try a different model.

## Topic Modeling - Attempt #1 (All Text)

In [1]:
# Let's read in our document-term matrix
import pandas as pd
import pickle

data = pd.read_pickle('dtm_stop.pkl')
data

,aaron,abaddon,abandon,abandoned,abandoning,abandons,abba,abel,abhor,abiathar,...,zealous,zebedee,zebulun,zechariah,zenas,zerah,zerubbabel,zeus,zion,zionhe
Matthew,0,0,0,0,0,0,0,1,0,0,...,0,6,2,1,0,1,2,0,1,0
Mark,0,0,0,0,0,0,1,0,0,1,...,0,4,0,0,0,0,0,0,0,0
Luke,1,0,0,0,0,0,0,1,0,0,...,0,1,0,12,0,0,1,0,0,0
John,0,0,1,0,0,1,0,0,0,0,...,0,1,0,0,0,0,0,0,1,0
Acts,1,0,2,3,0,0,0,0,0,0,...,1,0,0,0,0,0,0,2,0,0
Romans,0,0,0,1,0,0,1,0,2,0,...,1,0,0,0,0,0,0,0,1,1
1 Corinthians,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2 Corinthians,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Galatians,0,0,0,0,0,0,1,0,0,0,...,1,0,0,0,0,0,0,0,0,0
Ephesians,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [2]:
# Import the necessary modules for LDA with gensim
# Terminal / Anaconda Navigator: conda install -c conda-forge gensim
from gensim import matutils, models
import scipy.sparse

# import logging
# logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [3]:
# One of the required inputs is a term-document matrix
tdm = data.transpose()
tdm.head()

,Matthew,Mark,Luke,John,Acts,Romans,1 Corinthians,2 Corinthians,Galatians,Ephesians,...,Philemon,Hebrews,James,1 Peter,2 Peter,1 John,2 John,3 John,Jude,Revelation
aaron,0,0,1,0,1,0,0,0,0,0,...,0,3,0,0,0,0,0,0,0,0
abaddon,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
abandon,0,0,0,1,2,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
abandoned,0,0,0,0,3,1,0,1,0,0,...,0,0,0,0,0,0,0,0,2,0
abandoning,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0


In [4]:
# We're going to put the term-document matrix into a new gensim format, from df --> sparse matrix --> gensim corpus
sparse_counts = scipy.sparse.csr_matrix(tdm)
corpus = matutils.Sparse2Corpus(sparse_counts)

In [5]:
# Gensim also requires dictionary of the all terms and their respective location in the term-document matrix
cv = pickle.load(open('cv_stop.pkl', 'rb'))
id2word = dict((v, k) for k, v in cv.vocabulary_.items())

Now that we have the corpus (term-document matrix) and id2word (dictionary of location: term), we need to specify two other parameters - the number of topics and the number of passes. Let's start the number of topics at 2, see if the results make sense, and increase the number from there.

In [6]:
# Now that we have the corpus (term-document matrix) and id2word (dictionary of location: term),
# we need to specify two other parameters as well - the number of topics and the number of passes
# Here we consider LDA for num_topics = 2
lda = models.LdaModel(corpus=corpus, id2word=id2word, num_topics=2, passes=50)
lda.print_topics()

[(0,
  '0.031*"god" + 0.016*"christ" + 0.013*"lord" + 0.010*"jesus" + 0.007*"spirit" + 0.007*"things" + 0.007*"faith" + 0.007*"paul" + 0.006*"know" + 0.006*"people"'),
 (1,
  '0.024*"said" + 0.022*"jesus" + 0.011*"god" + 0.010*"man" + 0.009*"came" + 0.008*"son" + 0.008*"went" + 0.008*"father" + 0.007*"come" + 0.007*"people"')]

In [7]:
# LDA for num_topics = 3
lda = models.LdaModel(corpus=corpus, id2word=id2word, num_topics=3, passes=50)
lda.print_topics()

[(0,
  '0.024*"said" + 0.022*"jesus" + 0.012*"god" + 0.011*"man" + 0.009*"came" + 0.008*"son" + 0.008*"went" + 0.008*"father" + 0.008*"people" + 0.007*"lord"'),
 (1,
  '0.015*"god" + 0.009*"earth" + 0.009*"like" + 0.007*"angel" + 0.006*"come" + 0.006*"seven" + 0.006*"heaven" + 0.005*"saw" + 0.005*"throne" + 0.005*"beast"'),
 (2,
  '0.034*"god" + 0.020*"christ" + 0.014*"lord" + 0.012*"jesus" + 0.008*"things" + 0.008*"spirit" + 0.008*"faith" + 0.007*"know" + 0.007*"love" + 0.007*"law"')]

In [8]:
# LDA for num_topics = 4
lda = models.LdaModel(corpus=corpus, id2word=id2word, num_topics=4, passes=50)
lda.print_topics()

[(0,
  '0.000*"jesus" + 0.000*"god" + 0.000*"said" + 0.000*"lord" + 0.000*"people" + 0.000*"man" + 0.000*"things" + 0.000*"christ" + 0.000*"know" + 0.000*"come"'),
 (1,
  '0.028*"said" + 0.027*"jesus" + 0.011*"man" + 0.010*"came" + 0.010*"son" + 0.010*"god" + 0.009*"father" + 0.009*"went" + 0.008*"come" + 0.007*"people"'),
 (2,
  '0.028*"god" + 0.012*"lord" + 0.010*"christ" + 0.009*"paul" + 0.008*"spirit" + 0.008*"law" + 0.007*"people" + 0.006*"jesus" + 0.006*"things" + 0.006*"faith"'),
 (3,
  '0.028*"god" + 0.016*"christ" + 0.012*"lord" + 0.011*"jesus" + 0.007*"love" + 0.007*"things" + 0.007*"know" + 0.006*"like" + 0.006*"earth" + 0.006*"come"')]

We've tried modifying our parameters. Let's try modifying our terms list as well.

## Topic Modeling - Attempt #2 (Nouns Only)

One popular trick is to look only at terms that are from one part of speech (only nouns, only adjectives, etc.). Check out the UPenn tag set: https://www.ling.upenn.edu/courses/Fall_2003/ling001/penn_treebank_pos.html.

In [9]:
# Let's create a function to pull out nouns from a string of text
from nltk import word_tokenize, pos_tag

def nouns(text):
    """Given a string of text, tokenize the text and pull out only the nouns."""
    is_noun = lambda pos: pos[:2] == 'NN'
    tokenized = word_tokenize(text)
    all_nouns = [word for (word, pos) in pos_tag(tokenized) if is_noun(pos)] 
    return ' '.join(all_nouns)

In [10]:
# Read in the cleaned data, before the CountVectorizer step
pd.set_option('max_colwidth', 150)
data_clean = pd.read_pickle('data_clean.pkl')
data_clean

,book_text
Matthew,this is the record of the genealogy of jesus christ the son of david the son of abraham abraham was the father of isaac isaac the father of jac...
Mark,the beginning of the gospel of jesus christ the son of god as it is written in the prophet isaiah “look i am sending my messenger ahead of youwh...
Luke,now many have undertaken to compile an account of the things that have been fulfilled among us like the accounts passed on to us by those who w...
John,in the beginning was the word and the word was with god and the word was fully god the word was with god in the beginning all things were cre...
Acts,i wrote the former account theophilus about all that jesus began to do and teach until the day he was taken up to heaven after he had given ord...
Romans,from paul a slave of christ jesus called to be an apostle set apart for the gospel of god this gospel he promised beforehand through his prophe...
1 Corinthians,from paul called to be an apostle of christ jesus by the will of god and sosthenes our brother to the church of god that is in corinth to those...
2 Corinthians,from paul an apostle of christ jesus by the will of god and timothy our brother to the church of god that is in corinth with all the saints who a...
Galatians,from paul an apostle not from men nor by human agency but by jesus christ and god the father who raised him from the dead and all the brothers ...
Ephesians,from paul an apostle of christ jesus by the will of god to the saints in ephesus the faithful in christ jesus grace and peace to you from god o...


In [11]:
# Make sure necessary packages are available
import nltk
nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('averaged_perceptron_tagger')
nltk.download('averaged_perceptron_tagger_eng')

[nltk_data] Downloading package punkt to /home/senning/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     /home/senning/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/senning/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /home/senning/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!


True

In [12]:
# Apply the nouns function to the transcripts to filter only on nouns
data_nouns = pd.DataFrame(data_clean.book_text.apply(nouns))
data_nouns

,book_text
Matthew,record genealogy jesus son david son abraham abraham father isaac father jacob jacob father judah brothers father perez zerah tamar perez father h...
Mark,beginning gospel jesus son god prophet isaiah “ look i messenger youwho way voice shouting wilderness ‘ way lordmake paths ” wilderness baptizer b...
Luke,account things accounts eyewitnesses servants word beginning i things beginning account theophilus things reign king judea priest zechariah divisi...
John,beginning word word god word word god things thing life life light mankind shines darkness darkness man god name john witness light everyone light...
Acts,account theophilus jesus day orders holy spirit apostles apostles suffering proofs period matters kingdom god “ jerusalem father john water holy d...
Romans,slave christ jesus set apart gospel god gospel beforehand prophets holy son descendant david reference flesh sonofgodinpower holy spirit resurrect...
1 Corinthians,paul apostle christ jesus god brother church god corinth christ jesus saints place name lord jesus lord grace peace father lord jesus christ i god...
2 Corinthians,apostle christ jesus god brother church god corinth saints grace peace father lord jesus christ god father lord jesus father mercies god comfort t...
Galatians,men agency christ father brothers churches galatia grace peace father lord jesus christ sins age god father amen one grace christ gospel— gospel g...
Ephesians,apostle christ jesus god saints ephesus faithful christ jesus grace peace father lord jesus christ god father lord jesus christ blessing realms ch...


In [13]:
# Create a new document-term matrix using only nouns
from sklearn.feature_extraction import text
from sklearn.feature_extraction.text import CountVectorizer

# If we wanted to add any additional stop words, we could do that here
add_stop_words = []

stop_words = list(text.ENGLISH_STOP_WORDS.union(add_stop_words))

# Recreate a document-term matrix with only nouns
cvn = CountVectorizer(stop_words=stop_words)
data_cvn = cvn.fit_transform(data_nouns.book_text)
data_dtmn = pd.DataFrame(data_cvn.toarray(), columns=cvn.get_feature_names_out())
data_dtmn.index = data_nouns.index
data_dtmn

,aaron,abandons,abba,abel,abhor,abiathar,abijah,abilene,ability,ablaze,...,zeal,zealot,zebedee,zebulun,zechariah,zerah,zerubbabel,zeus,zion,zionhe
Matthew,0,0,0,1,0,0,0,0,1,0,...,0,1,6,2,1,1,2,0,1,0
Mark,0,0,0,0,0,1,0,0,0,0,...,0,1,4,0,0,0,0,0,0,0
Luke,1,0,0,1,0,0,1,1,0,0,...,0,1,1,0,10,0,1,0,0,0
John,0,1,0,0,0,0,0,0,0,0,...,1,0,1,0,0,0,0,0,1,0
Acts,0,0,0,0,0,0,0,0,1,0,...,0,1,0,0,0,0,0,2,0,0
Romans,0,0,1,0,1,0,0,0,0,0,...,2,0,0,0,0,0,0,0,1,1
1 Corinthians,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2 Corinthians,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
Galatians,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Ephesians,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [14]:
# Create the gensim corpus
corpusn = matutils.Sparse2Corpus(scipy.sparse.csr_matrix(data_dtmn.transpose()))

# Create the vocabulary dictionary
id2wordn = dict((v, k) for k, v in cvn.vocabulary_.items())

In [15]:
# Let's start with 2 topics
ldan = models.LdaModel(corpus=corpusn, num_topics=2, id2word=id2wordn, passes=50)
ldan.print_topics()

[(0,
  '0.040*"god" + 0.022*"lord" + 0.021*"christ" + 0.015*"jesus" + 0.013*"spirit" + 0.013*"things" + 0.012*"people" + 0.009*"brothers" + 0.009*"faith" + 0.009*"law"'),
 (1,
  '0.048*"jesus" + 0.023*"man" + 0.020*"son" + 0.018*"father" + 0.015*"god" + 0.015*"people" + 0.014*"disciples" + 0.012*"things" + 0.011*"lord" + 0.008*"day"')]

In [16]:
# Let's try topics = 3
ldan = models.LdaModel(corpus=corpusn, num_topics=3, id2word=id2wordn, passes=50)
ldan.print_topics()

[(0,
  '0.045*"god" + 0.030*"christ" + 0.025*"lord" + 0.020*"jesus" + 0.015*"things" + 0.014*"spirit" + 0.012*"faith" + 0.012*"law" + 0.010*"people" + 0.010*"brothers"'),
 (1,
  '0.000*"charges" + 0.000*"demetrius" + 0.000*"health" + 0.000*"pen" + 0.000*"diotrephes" + 0.000*"cooperation" + 0.000*"deceiver" + 0.000*"paper" + 0.000*"lady" + 0.000*"ink"'),
 (2,
  '0.038*"jesus" + 0.020*"god" + 0.020*"man" + 0.015*"son" + 0.015*"people" + 0.014*"father" + 0.013*"lord" + 0.011*"things" + 0.011*"disciples" + 0.009*"day"')]

In [17]:
# Let's try 4 topics
ldan = models.LdaModel(corpus=corpusn, num_topics=4, id2word=id2wordn, passes=50)
ldan.print_topics()

[(0,
  '0.047*"god" + 0.031*"christ" + 0.026*"lord" + 0.020*"jesus" + 0.015*"things" + 0.014*"spirit" + 0.013*"faith" + 0.012*"law" + 0.011*"people" + 0.010*"brothers"'),
 (1,
  '0.042*"jesus" + 0.024*"god" + 0.023*"man" + 0.018*"people" + 0.017*"lord" + 0.014*"father" + 0.012*"things" + 0.012*"son" + 0.012*"paul" + 0.011*"day"'),
 (2,
  '0.023*"god" + 0.022*"earth" + 0.016*"angel" + 0.013*"beast" + 0.012*"throne" + 0.010*"voice" + 0.009*"lamb" + 0.008*"city" + 0.008*"heaven" + 0.008*"people"'),
 (3,
  '0.041*"jesus" + 0.022*"son" + 0.018*"man" + 0.015*"father" + 0.014*"disciples" + 0.012*"people" + 0.012*"kingdom" + 0.010*"god" + 0.010*"things" + 0.010*"lord"')]

## Topic Modeling - Attempt #3 (Nouns and Adjectives)

In [18]:
# Let's create a function to pull out nouns from a string of text
def nouns_adj(text):
    """Given a string of text, tokenize the text and pull out only the nouns and adjectives."""
    is_noun_adj = lambda pos: pos[:2] == 'NN' or pos[:2] == 'JJ'
    tokenized = word_tokenize(text)
    nouns_adj = [word for (word, pos) in pos_tag(tokenized) if is_noun_adj(pos)] 
    return ' '.join(nouns_adj)

In [19]:
# Apply the nouns function to the book texts to filter only on nouns and adjectives
data_nouns_adj = pd.DataFrame(data_clean.book_text.apply(nouns_adj))
data_nouns_adj

,book_text
Matthew,record genealogy jesus son david son abraham abraham father isaac father jacob jacob father judah brothers father perez zerah tamar perez father h...
Mark,beginning gospel jesus son god prophet isaiah “ look i messenger youwho way voice shouting wilderness ‘ way lordmake paths ’ ” wilderness baptizer...
Luke,many account things accounts eyewitnesses servants word beginning good i things beginning orderly account most excellent theophilus certain things...
John,beginning word word god word word god things thing life life light mankind light shines darkness darkness man god name john witness light everyone...
Acts,i former account theophilus jesus day orders holy spirit apostles same apostles suffering alive many proofs fortyday period matters kingdom god “ ...
Romans,slave christ jesus apostle set apart gospel god gospel beforehand prophets holy son descendant david reference flesh sonofgodinpower holy spirit r...
1 Corinthians,paul apostle christ jesus god brother church god corinth christ jesus saints place name lord jesus lord ours grace peace father lord jesus christ ...
2 Corinthians,apostle christ jesus god brother church god corinth saints achaia grace peace father lord jesus christ god father lord jesus father mercies god co...
Galatians,apostle men human agency jesus christ father dead brothers churches galatia grace peace father lord jesus christ sins present evil age god father ...
Ephesians,apostle christ jesus god saints ephesus faithful christ jesus grace peace father lord jesus christ god father lord jesus christ spiritual blessing...


In [20]:
# Create a new document-term matrix using only nouns and adjectives, also remove common words with max_df
cvna = CountVectorizer(stop_words=stop_words, max_df=.8)
data_cvna = cvna.fit_transform(data_nouns_adj.book_text)
data_dtmna = pd.DataFrame(data_cvna.toarray(), columns=cvna.get_feature_names_out())
data_dtmna.index = data_nouns_adj.index
data_dtmna

,aaron,abaddon,abandons,abba,abel,abhor,abiathar,abijah,abilene,ability,...,zealot,zealous,zebedee,zebulun,zechariah,zerah,zerubbabel,zeus,zion,zionhe
Matthew,0,0,0,0,1,0,0,1,0,1,...,1,0,6,2,1,1,2,0,1,0
Mark,0,0,0,0,0,0,1,0,0,0,...,1,0,4,0,0,0,0,0,0,0
Luke,1,0,0,0,1,0,0,1,1,0,...,1,0,1,0,11,0,1,0,0,0
John,0,0,1,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,1,0
Acts,0,0,0,0,0,0,0,0,0,1,...,1,1,0,0,0,0,0,2,0,0
Romans,0,0,0,1,0,1,0,0,0,0,...,0,1,0,0,0,0,0,0,1,1
1 Corinthians,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2 Corinthians,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Galatians,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
Ephesians,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [21]:
# Create the gensim corpus
corpusna = matutils.Sparse2Corpus(scipy.sparse.csr_matrix(data_dtmna.transpose()))

# Create the vocabulary dictionary
id2wordna = dict((v, k) for k, v in cvna.vocabulary_.items())

In [22]:
# Let's start with 2 topics
ldana = models.LdaModel(corpus=corpusna, num_topics=2, id2word=id2wordna, passes=50)
ldana.print_topics()

[(0,
  '0.019*"man" + 0.010*"disciples" + 0.008*"day" + 0.007*"house" + 0.006*"peter" + 0.006*"heaven" + 0.006*"paul" + 0.006*"kingdom" + 0.006*"earth" + 0.006*"place"'),
 (1,
  '0.012*"law" + 0.008*"body" + 0.008*"grace" + 0.007*"world" + 0.007*"glory" + 0.006*"gospel" + 0.006*"righteousness" + 0.006*"holy" + 0.006*"day" + 0.006*"sin"')]

In [23]:
# Let's try 3 topics
ldana = models.LdaModel(corpus=corpusna, num_topics=3, id2word=id2wordna, passes=50)
ldana.print_topics()

[(0,
  '0.015*"law" + 0.010*"grace" + 0.009*"gospel" + 0.008*"righteousness" + 0.007*"world" + 0.007*"glory" + 0.006*"sin" + 0.006*"body" + 0.006*"holy" + 0.006*"flesh"'),
 (1,
  '0.023*"man" + 0.013*"disciples" + 0.009*"day" + 0.008*"peter" + 0.008*"house" + 0.007*"paul" + 0.007*"kingdom" + 0.007*"crowd" + 0.007*"law" + 0.007*"men"'),
 (2,
  '0.011*"earth" + 0.007*"angel" + 0.006*"great" + 0.006*"body" + 0.006*"throne" + 0.006*"blood" + 0.006*"world" + 0.005*"beast" + 0.005*"angels" + 0.005*"glory"')]

In [24]:
# Let's try 4 topics
ldana = models.LdaModel(corpus=corpusna, num_topics=4, id2word=id2wordna, passes=50)
ldana.print_topics()

[(0,
  '0.024*"man" + 0.014*"disciples" + 0.008*"kingdom" + 0.008*"day" + 0.007*"house" + 0.007*"crowd" + 0.007*"world" + 0.007*"law" + 0.007*"heaven" + 0.007*"place"'),
 (1,
  '0.016*"law" + 0.011*"grace" + 0.010*"body" + 0.009*"gospel" + 0.008*"glory" + 0.007*"righteousness" + 0.006*"sin" + 0.006*"flesh" + 0.006*"holy" + 0.006*"death"'),
 (2,
  '0.016*"world" + 0.009*"fellow" + 0.007*"christian" + 0.007*"sin" + 0.006*"commandment" + 0.006*"resides" + 0.006*"little" + 0.005*"dear" + 0.005*"darkness" + 0.005*"friends"'),
 (3,
  '0.014*"paul" + 0.010*"man" + 0.010*"earth" + 0.009*"day" + 0.008*"angel" + 0.008*"great" + 0.008*"holy" + 0.008*"city" + 0.008*"men" + 0.006*"jews"')]

## Identify Topics in Each Document

Out of the 9 topic models we looked at, the nouns and adjectives, 3 topic one made the most sense. So let's pull that down here and run it through some more iterations to get more fine-tuned topics.

In [25]:
# Our final LDA model (for now)
# Set random_state so we get the same list each time
ldana = models.LdaModel(random_state=42, corpus=corpusna, num_topics=3, id2word=id2wordna, passes=250)
ldana.print_topics()

[(0,
  '0.016*"law" + 0.011*"body" + 0.010*"grace" + 0.009*"gospel" + 0.009*"world" + 0.007*"righteousness" + 0.007*"glory" + 0.007*"man" + 0.006*"sin" + 0.006*"flesh"'),
 (1,
  '0.012*"earth" + 0.008*"angel" + 0.007*"great" + 0.006*"throne" + 0.006*"beast" + 0.006*"angels" + 0.006*"heaven" + 0.006*"day" + 0.006*"blood" + 0.006*"holy"'),
 (2,
  '0.022*"man" + 0.012*"disciples" + 0.009*"day" + 0.008*"house" + 0.008*"peter" + 0.007*"paul" + 0.007*"kingdom" + 0.007*"law" + 0.007*"crowd" + 0.007*"men"')]

These three topics look pretty decent. Let's settle on these for now
- Topic 0: Law and Grace (law, body, grace, gospel, righteousness, sin, holy, death)
- Topic 1: Heaven and Earth (earth, angel, beast, throne, heaven, city)
- Topic 2: Humankined (man, disciples, house, peter, paul, kingdom, crowd)

In [26]:
# Let's take a look at which topics each book contains
corpus_transformed = ldana[corpusna]
list(zip([max(x)[0] for x in corpus_transformed], data_dtmna.index))

[(2, 'Matthew'),
 (2, 'Mark'),
 (2, 'Luke'),
 (2, 'John'),
 (2, 'Acts'),
 (0, 'Romans'),
 (0, '1 Corinthians'),
 (1, '2 Corinthians'),
 (0, 'Galatians'),
 (0, 'Ephesians'),
 (0, 'Philippians'),
 (0, 'Colossians'),
 (0, '1 Thessalonians'),
 (0, '2 Thessalonians'),
 (1, '1 Timothy'),
 (1, '2 Timothy'),
 (0, 'Titus'),
 (0, 'Philemon'),
 (1, 'Hebrews'),
 (1, 'James'),
 (1, '1 Peter'),
 (1, '2 Peter'),
 (0, '1 John'),
 (2, '2 John'),
 (0, '3 John'),
 (1, 'Jude'),
 (1, 'Revelation')]

### For a first pass of LDA:
- Topic 0: Law and Grace (law, body, grace, gospel, righteousness, sin, holy, death)
- Topic 1: Heaven and Earth (earth, angel, beast, throne, heaven, city)
- Topic 2: Humankined (man, disciples, house, peter, paul, kingdom, crowd)
  
|Law and Grace  |Heaven and Earth|Humankind|
|---------------|----------------|---------|
|Romans         |2 Corinthians   |Matthew |
|1 Corinthians  |1 Timothy       |Mark    |
|Galations      |2 Timothy       |Luke    |
|Ephesians      |Hebrews         |John    |
|Philippians    |James           |Acts    |
|Colossians     |1 Peter         |2 John  |
|1 Thessalonians|2 Peter         |        |
|2 Thessalonians|Jude            |        |
|Titus          |Revelation      |        |
|Philemon       |                |        |
|1 John         |                |        |
|3 John         |                |        |

## Additional Exercises

1. Try further modifying the parameters of the topic models above and see if you can get better topics.
2. Create a new topic model that includes terms from a different [part of speech](https://www.ling.upenn.edu/courses/Fall_2003/ling001/penn_treebank_pos.html) and see if you can get better topics.